# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
cities_output_csv = "../WeatherPy/output_data/cities.csv"
city_results = pd.read_csv(cities_output_csv)
city_results.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,73.92,86,95,24.00,PF,1604281811
1,Ushuaia,-54.80,-68.30,57.20,54,75,3.36,AR,1604281801
2,Redwater,33.36,-94.25,57.20,47,1,3.36,US,1604282110
3,New Norfolk,-42.78,147.06,77.00,30,100,3.00,AU,1604282055
4,Vanimo,-2.67,141.30,86.52,64,76,6.80,PG,1604281875


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
#Remove NA results from dataframe
city_results = city_results.dropna()

In [29]:
gmaps.configure(api_key=g_key)

In [30]:
#Set variables
locations = city_results[["Lat","Lng"]]

In [31]:
humidity = city_results["Humidity"]

In [32]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
hotel_df = city_results[(city_results["Humidity"] <= 35) & (city_results["Wind Speed"] < 5) & (city_results["Max Temp"] > 55) & (city_results["Max Temp"] < 70 ) & (city_results["Cloudiness"] < 5)]

hotel_df = hotel_df.reset_index(drop = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Gorakhpur,29.45,75.68,63.57,27,0,2.46,IN,1604281924
1,Mānsa,29.98,75.38,61.86,28,0,2.17,IN,1604282115
2,Tiznit Province,29.58,-9.50,68.04,28,0,2.82,MA,1604282117
3,Iranshahr,27.20,60.68,57.22,25,0,4.63,IR,1604282145
4,Elko,41.00,-115.50,59.00,16,1,4.70,US,1604282153
5,Faya,18.39,42.45,57.20,34,0,2.17,SA,1604281886
6,Guozhen,34.37,107.36,59.11,31,0,2.53,CN,1604282246


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gorakhpur,29.45,75.68,63.57,27,0,2.46,IN,1604281924,
1,Mānsa,29.98,75.38,61.86,28,0,2.17,IN,1604282115,
2,Tiznit Province,29.58,-9.50,68.04,28,0,2.82,MA,1604282117,
3,Iranshahr,27.20,60.68,57.22,25,0,4.63,IR,1604282145,
4,Elko,41.00,-115.50,59.00,16,1,4.70,US,1604282153,


In [35]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params).json()
    #pprint(name_address)



    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "Not Found"
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gorakhpur,29.45,75.68,63.57,27,0,2.46,IN,1604281924,Not Found
1,Mānsa,29.98,75.38,61.86,28,0,2.17,IN,1604282115,Hotel The City Crown
2,Tiznit Province,29.58,-9.50,68.04,28,0,2.82,MA,1604282117,Not Found
3,Iranshahr,27.20,60.68,57.22,25,0,4.63,IR,1604282145,هتل خنونه سرزه
4,Elko,41.00,-115.50,59.00,16,1,4.70,US,1604282153,Not Found
5,Faya,18.39,42.45,57.20,34,0,2.17,SA,1604281886,Not Found
6,Guozhen,34.37,107.36,59.11,31,0,2.53,CN,1604282246,Lily Garden Hotel


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))